In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L2\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['COAST.csv', 'EAST.csv', 'FWEST.csv', 'NCENT.csv', 'NORTH.csv', 'SCENT.csv', 'SOUTH.csv', 'WEST.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[0]))

,Net,Temprature
0,9844.200268,6.8
1,9653.456269,7.1
2,9502.299659,7.1
3,9474.478036,7.0
4,9554.370489,6.5
...,...,...
43818,11109.257856,14.8
43819,10846.500062,14.3
43820,10592.539874,14.1
43821,10304.624870,13.7


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[0]

zoneData = pd.read_csv(os.path.join(path, fileName))
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_COAST,Temp_COAST
0,9844.200268,6.8
1,9844.200268,6.8
2,9653.456269,7.1
3,9502.299659,7.1
4,9474.478036,7.0
...,...,...
43819,11109.257856,14.8
43820,10846.500062,14.3
43821,10592.539874,14.1
43822,10304.624870,13.7


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_COAST,Temp_COAST,Lag-2,Lag-7
0,10882.679600,11.0,10663.854100,9844.200268
1,10671.664509,11.1,10311.614249,9844.200268
2,10685.756321,11.2,10153.955297,9653.456269
3,10816.410606,9.2,10107.504102,9502.299659
4,11045.887947,7.9,10198.666936,9474.478036
...,...,...,...,...
43651,11109.257856,14.8,10520.134445,10523.664434
43652,10846.500062,14.3,10438.584114,10335.992990
43653,10592.539874,14.1,10306.359839,10144.751404
43654,10304.624870,13.7,10005.692012,9876.787820


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 17s 12ms/step - loss: 0.0137 - mape: 3507.9475 - mae: 0.0880 - val_loss: 0.0072 - val_mape: 21.9190 - val_mae: 0.0654
Epoch 2/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0082 - mape: 2300.4463 - mae: 0.0690 - val_loss: 0.0149 - val_mape: 26.1825 - val_mae: 0.0935
Epoch 3/100
1105/1105 [==============================] - 11s 10ms/step - loss: 0.0061 - mape: 1885.8324 - mae: 0.0593 - val_loss: 0.0434 - val_mape: 42.6299 - val_mae: 0.1502
Epoch 4/100
1105/1105 [==============================] - 11s 10ms/step - loss: 0.0056 - mape: 1297.5703 - mae: 0.0569 - val_loss: 0.0325 - val_mape: 34.4074 - val_mae: 0.1316
Epoch 5/100
1105/1105 [==============================] - 11s 10ms/step - loss: 0.0052 - mape: 1612.3761 - mae: 0.0548 - val_loss: 0.0472 - val_mape: 41.1532 - val_mae: 0.1542
Epoch 6/100
1105/1105 [==============================] - 11s 10ms/step - loss: 0.0051 - mape: 1537.8566 - mae: 0.0540 - val_l

Epoch 48/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0028 - mape: 971.7347 - mae: 0.0389 - val_loss: 0.0406 - val_mape: 34.4620 - val_mae: 0.1423
Epoch 49/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0028 - mape: 1164.5428 - mae: 0.0388 - val_loss: 0.0426 - val_mape: 36.5727 - val_mae: 0.1495
Epoch 50/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0027 - mape: 1096.5574 - mae: 0.0385 - val_loss: 0.0440 - val_mape: 37.7387 - val_mae: 0.1525
Epoch 51/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0027 - mape: 942.1529 - mae: 0.0382 - val_loss: 0.0445 - val_mape: 36.5252 - val_mae: 0.1511
Epoch 52/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0027 - mape: 907.6771 - mae: 0.0379 - val_loss: 0.0398 - val_mape: 34.8436 - val_mae: 0.1430
Epoch 53/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0026 - mape: 994.0379 - mae: 0.0379 - val

Epoch 95/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0016 - mape: 906.8042 - mae: 0.0295 - val_loss: 0.0358 - val_mape: 33.8896 - val_mae: 0.1355
Epoch 96/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0015 - mape: 883.1916 - mae: 0.0292 - val_loss: 0.0393 - val_mape: 35.4388 - val_mae: 0.1428
Epoch 97/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0016 - mape: 981.9465 - mae: 0.0298 - val_loss: 0.0365 - val_mape: 33.4843 - val_mae: 0.1372
Epoch 98/100
1105/1105 [==============================] - 12s 11ms/step - loss: 0.0015 - mape: 1001.3812 - mae: 0.0290 - val_loss: 0.0398 - val_mape: 35.2310 - val_mae: 0.1428
Epoch 99/100
1105/1105 [==============================] - 12s 10ms/step - loss: 0.0015 - mape: 849.4227 - mae: 0.0288 - val_loss: 0.0400 - val_mape: 35.3490 - val_mae: 0.1433
Epoch 100/100
1105/1105 [==============================] - 11s 10ms/step - loss: 0.0015 - mape: 1107.3884 - mae: 0.0285 - va

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1167.590999364853


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 6ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.16729274, 0.13779773, 0.13813841, ..., 0.33452636, 0.3005378 ,
       0.26555336], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.34604847, 0.32097241, 0.30230427, ..., 0.25608669, 0.23395411,
       0.21205685])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,11762.821446,9437.450195
1,11436.615685,9053.759766
2,11193.768371,9058.192383
3,11255.497390,9338.033203
4,11660.651512,9795.186523
...,...,...
4363,11109.257856,12171.953125
4364,10846.500062,11993.094727
4365,10592.539874,11612.934570
4366,10304.624870,11170.789062


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

12.83326038828963